In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


1. 나눔 폰트 설치

In [ ]:
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 34 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 1s (7,435 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 126102 files and dire

2. Colab 런타임 다시 시작
상단탭에서 [런타임] > [런타임 다시 시작] 클릭
3. matplotlib 폰트 변경
```
import matplotlib.pyplot as plt

plt.rcParams['font.family'] = 'NanumGothic'
```
출처: https://developnote.tistory.com/165 [범범범즈의 개발 노트:티스토리]

코랩에서 상대 경로를 사용하여 파일을 불러오는 방법

In [ ]:
# 1. 먼저 현재 작업 디렉토리를 확인합니다:
import os
print(os.getcwd())
# 2. 필요한 경우 작업 디렉토리를 변경합니다:
os.chdir('/content/drive/MyDrive/Colab Notebooks/아이펠/DLThon/')#←('/content')  # 코랩의 기본 디렉토리로 변경
# 3. 그 다음 상대 경로를 사용하여 파일을 불러올 수 있습니다:

/content


여기까지 하고 시작

---

# preprocessing.py 의 기존 코드입니다.

In [ ]:
import re

# 1. 텍스트 정제 함수들

def remove_stopwords(tokens, stopword_list):
    stopword_set = stopword_list if isinstance(stopword_list, set) else set(stopword_list)
    return [token for token in tokens if token not in stopword_set]

def clean_text(text):
    text = re.sub(r"[^\w\s가-힣]", " ", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

def normalize_repetitions(text, repeat_limit=2):
    # 문자 반복 (예: ㅋㅋㅋㅋ → ㅋㅋ)
    text = re.sub(r'(.)\1{2,}', lambda m: m.group(1) * repeat_limit, text)

    # 음절 반복 (예: 하하하하 → 하하)
    text = re.sub(r'((..))\1{1,}', lambda m: m.group(1) * repeat_limit, text)

    return text


# 2. 텍스트 정제

def tokenize_and_clean_text(text, stopword_list=None, repeat_limit=2):
    text = normalize_repetitions(text, repeat_limit=repeat_limit)
    text = clean_text(text)
    tokens = text.split()
    if stopword_list:
        tokens = remove_stopwords(tokens, stopword_list)
    return tokens


# 3. 한줄 단위 전처리
def preprocess_conversation_lines(
    text,
    stopwords=None,
    use_silence=False,
    speaker_token="[UTTER]",
    repeat_limit=2
):
    lines = text.strip().split('\n')
    results = []

    for line in lines:
        if not line.strip():
            processed = ["[SILENCE]"] if use_silence else []
        else:
            processed = tokenize_and_clean_text(line, stopword_list=stopwords, repeat_limit=repeat_limit)
            if not processed:
                processed = ["[SILENCE]"] if use_silence else []
            else:
                processed = [speaker_token] + processed

        if processed:
            results.append(" ".join(processed).strip())

    return results


# 4. 여러줄을 한 줄로 flatten 함수
def flatten_utterances(utterance_tokens_list, sep_token=" "):
    return sep_token.join(utterance_tokens_list).strip()


# 5.  전체 전처리 파이프라인
def preprocess(
    text,
    stopwords=None,
    speaker_token="[UTTER]",
    use_silence=True,
    sep_token=" ",
    repeat_limit=2
):
    """
    전체 전처리 통합 함수
    """
    utterance_tokens = preprocess_conversation_lines(
        text,
        stopwords=stopwords,
        use_silence=use_silence,
        speaker_token=speaker_token,
        repeat_limit=repeat_limit
    )
    return flatten_utterances(utterance_tokens, sep_token=sep_token)


---
#### KoELECTRA 모델 구조와 파라미터를 확인

In [ ]:
# # 필요한 라이브러리 설치
# !pip install transformers pandas scikit-learn
# !pip install torch

# 필요한 라이브러리 임포트
import torch
import pandas as pd
from transformers import ElectraModel, ElectraTokenizer, ElectraForSequenceClassification
# AdamW는 이제 torch.optim에서 가져와야 합니다
from torch.optim import AdamW
# get_linear_schedule_with_warmup은 여전히 transformers에 있습니다
from transformers import get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
import numpy as np
import random
import os
from tqdm import tqdm

# 재현성을 위한 시드 설정
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

# KoELECTRA 모델 및 토크나이저 로드
model_name = "monologg/koelectra-base-v3-discriminator"
tokenizer = ElectraTokenizer.from_pretrained(model_name)

# 분류를 위한 모델 로드 (클래스 개수를 5로 설정)
model = ElectraForSequenceClassification.from_pretrained(model_name, num_labels=5)

# GPU 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 모델 구조 확인
print("=== Model Architecture ===")
print(model)

# 모델 파라미터 이름 확인
print("\n=== Parameter Names ===")
for name, param in model.named_parameters():
    print(f"{name}: {param.shape}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/263k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/467 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/452M [00:00<?, ?B/s]

Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


=== Model Architecture ===
ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(35000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
 

---

#### 1. 줄 바꿈 표시 X, 불용어 X
- KoELECTRA 모델의 인코더 부분을 고정(freeze)하고 분류기 부분만 학습
  - 전이 학습(transfer learning)의 한 종류, 사전 훈련된 언어 모델의 표현력을 유지하면서 특정 태스크에 적응시키는 효율적인 방법
- 클래스 불균형 문제
  - 분류 보고서에서 볼 수 있듯이, 모델이 대부분 '일반 대화'로 예측하고 있습니다(recall 0.99).
  - 클래스 가중치 적용: 클래스별 샘플 수에 반비례하는 가중치를 계산하여 적용
    - **성능 향상 있었음**

- 비교군으로 성능 향상으로 위해 학습률을 낮추고, Warmup 비율 설정

In [ ]:
# 필요한 라이브러리 설치
!pip install transformers pandas scikit-learn
!pip install torch

import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score, confusion_matrix
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import ElectraTokenizer, ElectraForSequenceClassification
from transformers import get_linear_schedule_with_warmup
from torch.optim import AdamW
from tqdm import tqdm
import random
import matplotlib.pyplot as plt
import seaborn as sns

# 시드 고정
def set_seed(seed_value=42):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)
    torch.backends.cudnn.deterministic = True

set_seed(42)

# 설정값
MODEL_NAME = "monologg/koelectra-base-v3-discriminator"
MAX_LEN = 128
BATCH_SIZE = 32
EPOCHS = 100  # 에폭 100으로 설정
EARLY_STOPPING_PATIENCE = 10  # Early Stopping 인내심 증가
LEARNING_RATE = 5e-6  # 학습률 낮춤
TRAIN_FILE = "data/train_preprocessed_1.csv"
TEXT_COL = "text"
LABEL_COL = "class"
LABEL_DICT = {'협박 대화': 0, '갈취 대화': 1, '직장 내 괴롭힘 대화': 2, '기타 괴롭힘 대화': 3, '일반 대화': 4}
ID_TO_LABEL = {v: k for k, v in LABEL_DICT.items()}
GPU_NUM = 0
TEST_FILE = "data/test.csv"

# 1. 데이터 로드 및 전처리
print("데이터 로드 중...")
df = pd.read_csv(TRAIN_FILE)

# 라벨 매핑
df[LABEL_COL] = df[LABEL_COL].map(LABEL_DICT)

# 데이터 분할
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42, stratify=df[LABEL_COL])
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df[LABEL_COL])

print(f"학습 데이터 크기: {len(train_df)}")
print(f"검증 데이터 크기: {len(val_df)}")
print(f"테스트 데이터 크기: {len(test_df)}")

# 클래스별 데이터 개수 확인
print("학습 데이터의 클래스 분포:")
print(train_df[LABEL_COL].value_counts())

# 2. Dataset 클래스 정의
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=True,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'token_type_ids': encoding['token_type_ids'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# 3. 데이터 로더 생성
def create_data_loader(df, tokenizer, max_len, batch_size):
    dataset = TextClassificationDataset(
        texts=df[TEXT_COL].values,
        labels=df[LABEL_COL].values,
        tokenizer=tokenizer,
        max_len=max_len
    )

    return DataLoader(
        dataset,
        batch_size=batch_size,
        num_workers=2
    )

# 4. 토크나이저 및 모델 로드
print(f"모델 '{MODEL_NAME}' 로드 중...")
tokenizer = ElectraTokenizer.from_pretrained(MODEL_NAME)
model = ElectraForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=len(LABEL_DICT))

# ELECTRA 인코더 부분 freeze
for param in model.electra.parameters():
    param.requires_grad = False

# classifier 부분만 학습 가능하게 설정
for param in model.classifier.parameters():
    param.requires_grad = True

# 학습 가능한 파라미터 확인 (디버깅용)
trainable_params = 0
all_param = 0
for name, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
        trainable_params += param.numel()
        print(f"✅ 학습 대상: {name}")
print(f"🔢 학습 대상 파라미터 수: {trainable_params:,} / 전체 파라미터 수: {all_param:,} ({trainable_params/all_param:.2%})")

# 5. 데이터 로더 생성
train_data_loader = create_data_loader(train_df, tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(val_df, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = create_data_loader(test_df, tokenizer, MAX_LEN, BATCH_SIZE)

# 6. 모델 학습 함수 (학습률 표시 추가)
def train_model(model, data_loader, optimizer, scheduler, device, current_lr):
    model.train()
    losses = []

    progress_bar = tqdm(data_loader, desc=f"학습 중 (학습률: {current_lr:.8f})")

    for batch in progress_bar:
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            labels=labels
        )

        loss = outputs.loss
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()
        scheduler.step()

        losses.append(loss.item())
        progress_bar.set_postfix({"loss": f"{np.mean(losses):.4f}"})

    return np.mean(losses)

# 7. 모델 평가 함수
def evaluate_model(model, data_loader, device):
    model.eval()
    losses = []
    predictions = []
    real_labels = []

    with torch.no_grad():
        for batch in tqdm(data_loader, desc="평가 중"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids,
                labels=labels
            )

            _, preds = torch.max(outputs.logits, dim=1)

            losses.append(outputs.loss.item())
            predictions.extend(preds.detach().cpu().numpy())
            real_labels.extend(labels.detach().cpu().numpy())

    return np.mean(losses), predictions, real_labels

# 8. 테스트 세트 예측 함수
def predict_test(model, data_loader, device):
    model.eval()
    predictions = []

    with torch.no_grad():
        for batch in tqdm(data_loader, desc="테스트 예측 중"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids
            )

            _, preds = torch.max(outputs.logits, dim=1)
            predictions.extend(preds.detach().cpu().numpy())

    return predictions

# 9. 혼동 행렬 시각화 함수
def plot_confusion_matrix(cm, classes):
    # 한글 폰트 설정
    import matplotlib.font_manager as fm

    try:
        # 방법 1: 나눔 글꼴 설치 및 사용
        !apt-get update -qq
        !apt-get install fonts-nanum -qq
        plt.rc('font', family='NanumGothic')
    except:
        try:
            # 방법 2: 한글 레이블을 영어로 변환
            classes = ['Threat', 'Extortion', 'Workplace Harassment', 'Other Harassment', 'Normal Conversation']
        except:
            pass

    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
    plt.xlabel('예측값')
    plt.ylabel('실제값')
    plt.title('혼동 행렬')
    plt.savefig('confusion_matrix.png')
    plt.close()

# 10. 학습 메인 함수
def train():
    # GPU 사용 설정
    device = torch.device(f"cuda:{GPU_NUM}" if torch.cuda.is_available() else "cpu")
    print(f"사용 중인 디바이스: {device}")
    model.to(device)

    # 옵티마이저 및 스케줄러 설정
    optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)

    # Warmup 비율 설정 (전체 훈련 스텝의 10%)
    warmup_ratio = 0.1
    total_steps = len(train_data_loader) * EPOCHS
    warmup_steps = int(total_steps * warmup_ratio)

    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=warmup_steps,
        num_training_steps=total_steps
    )

    # 학습 시작
    best_val_f1 = 0
    patience_counter = 0  # Early Stopping을 위한 카운터

    # 에폭별 F1 점수 및 손실 기록
    train_losses = []
    val_losses = []
    val_f1_scores = []

    for epoch in range(EPOCHS):
        # 현재 학습률 가져오기
        current_lr = optimizer.param_groups[0]['lr']

        print(f"\nEpoch {epoch + 1}/{EPOCHS}")

        # 학습
        train_loss = train_model(model, train_data_loader, optimizer, scheduler, device, current_lr)
        print(f"Train Loss: {train_loss:.4f}, 현재 학습률: {current_lr:.8f}")
        train_losses.append(train_loss)

        # 검증
        val_loss, val_predictions, val_labels = evaluate_model(model, val_data_loader, device)
        val_f1 = f1_score(val_labels, val_predictions, average='macro')
        val_losses.append(val_loss)
        val_f1_scores.append(val_f1)

        print(f"Validation Loss: {val_loss:.4f}")
        print(f"Validation Macro F1: {val_f1:.4f}")

        # 최고 성능 모델 저장 및 Early Stopping 로직
        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            torch.save(model.state_dict(), "best_model.pth")
            print(f"모델 저장됨: best_model.pth (F1: {val_f1:.4f})")
            patience_counter = 0  # 성능이 향상되었으므로 카운터 초기화
        else:
            patience_counter += 1  # 성능이 향상되지 않았으므로 카운터 증가
            print(f"성능 향상 없음: {patience_counter}/{EARLY_STOPPING_PATIENCE}")

            # Early Stopping 조건 확인
            if patience_counter >= EARLY_STOPPING_PATIENCE:
                print(f"\nEarly Stopping! {EARLY_STOPPING_PATIENCE}번의 에폭 동안 성능 향상 없음.")
                break

    # 학습 곡선 시각화
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Loss Curves')

    plt.subplot(1, 2, 2)
    plt.plot(val_f1_scores, label='Validation F1')
    plt.xlabel('Epoch')
    plt.ylabel('F1 Score')
    plt.legend()
    plt.title('F1 Score Curve')

    plt.tight_layout()
    plt.savefig('learning_curves.png')
    plt.close()

    # 최고 성능 모델 로드
    model.load_state_dict(torch.load("best_model.pth"))

    # 테스트 세트 평가
    test_loss, test_predictions, test_labels = evaluate_model(model, test_data_loader, device)
    test_f1 = f1_score(test_labels, test_predictions, average='macro')

    print("\n테스트 결과:")
    print(f"Test Loss: {test_loss:.4f}")
    print(f"Test Macro F1: {test_f1:.4f}")

    # 분류 보고서 출력
    class_names = [ID_TO_LABEL[i] for i in range(len(LABEL_DICT))]
    print("\n분류 보고서:")
    print(classification_report(test_labels, test_predictions, target_names=class_names))

    # 혼동 행렬 시각화
    cm = confusion_matrix(test_labels, test_predictions)
    plot_confusion_matrix(cm, class_names)

    return model

# 11. 실제 테스트 데이터 예측 및 제출 파일 생성
def predict_and_save():
    # 테스트 파일 로드
    test_data = pd.read_csv(TEST_FILE)

    # Dataset 생성
    class TestDataset(Dataset):
        def __init__(self, texts, tokenizer, max_len):
            self.texts = texts
            self.tokenizer = tokenizer
            self.max_len = max_len

        def __len__(self):
            return len(self.texts)

        def __getitem__(self, idx):
            text = str(self.texts[idx])

            encoding = self.tokenizer.encode_plus(
                text,
                add_special_tokens=True,
                max_length=self.max_len,
                return_token_type_ids=True,
                padding='max_length',
                truncation=True,
                return_attention_mask=True,
                return_tensors='pt'
            )

            return {
                'text': text,
                'input_ids': encoding['input_ids'].flatten(),
                'attention_mask': encoding['attention_mask'].flatten(),
                'token_type_ids': encoding['token_type_ids'].flatten(),
            }

    # 테스트 데이터 로더 생성
    test_dataset = TestDataset(
        texts=test_data['text'].values,
        tokenizer=tokenizer,
        max_len=MAX_LEN
    )

    test_data_loader = DataLoader(
        test_dataset,
        batch_size=BATCH_SIZE,
        num_workers=2
    )

    # GPU 설정
    device = torch.device(f"cuda:{GPU_NUM}" if torch.cuda.is_available() else "cpu")

    # 최고 성능 모델 로드
    model = ElectraForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=len(LABEL_DICT))
    model.load_state_dict(torch.load("best_model.pth"))
    model.to(device)

    # 예측
    model.eval()
    predictions = []

    with torch.no_grad():
        for batch in tqdm(test_data_loader, desc="테스트 예측 중"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids
            )

            _, preds = torch.max(outputs.logits, dim=1)
            predictions.extend(preds.detach().cpu().numpy())

    # 예측 결과 라벨로 변환
    label_predictions = [ID_TO_LABEL[p] for p in predictions]

    # 제출 파일 생성
    submission = pd.DataFrame({
        'idx': test_data['idx'],
        'class': label_predictions
    })

    submission.to_csv('submission.csv', index=False)
    print(f"제출 파일 생성 완료: submission.csv")

# 메인 실행
if __name__ == "__main__":
    # 모델 학습
    model = train()

    # 테스트 데이터 예측 및 제출 파일 생성
    predict_and_save()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 129.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 104.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 77.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvji

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/263k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/467 [00:00<?, ?B/s]

...

Validation Loss: 1.2453
Validation Macro F1: 0.5113
성능 향상 없음: 10/10

Early Stopping! 10번의 에폭 동안 성능 향상 없음.


평가 중: 100%|██████████| 24/24 [00:01<00:00, 16.86it/s]



테스트 결과:
Test Loss: 1.2149
Test Macro F1: 0.4708

분류 보고서:
              precision    recall  f1-score   support

       협박 대화       0.50      0.26      0.34       134
       갈취 대화       0.52      0.40      0.45       146
 직장 내 괴롭힘 대화       0.42      0.34      0.38       145
   기타 괴롭힘 대화       0.40      0.47      0.43       152
       일반 대화       0.61      0.97      0.75       169

    accuracy                           0.51       746
   macro avg       0.49      0.49      0.47       746
weighted avg       0.49      0.51      0.48       746

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
테스트 예측 중: 100%|██████████| 16/16 [00:01<00:00, 15.33it/s]


제출 파일 생성 완료: submission.csv


- 성능 향상

In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score, confusion_matrix
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import ElectraTokenizer, ElectraForSequenceClassification
from transformers import get_linear_schedule_with_warmup
from torch.optim import AdamW
from tqdm import tqdm
import random
import matplotlib.pyplot as plt
import seaborn as sns

# 시드 고정
def set_seed(seed_value=42):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)
    torch.backends.cudnn.deterministic = True

set_seed(42)

# 설정값
MODEL_NAME = "monologg/koelectra-base-v3-discriminator"
MAX_LEN = 128
BATCH_SIZE = 32
EPOCHS = 100  # 에폭 100으로 유지
SCHEDULER_EPOCHS = 5  # 스케줄러용 에폭 (원래 설정 유지)
EARLY_STOPPING_PATIENCE = 5  # 연속 5번의 에폭 동안 성능이 향상되지 않으면 학습 중단
LEARNING_RATE = 1.5e-5  # 8e-6에서 증가
TRAIN_FILE = "data/train_preprocessed_1.csv" # 줄 바꿈 표시 x, 불용어 X
TEXT_COL = "text"
LABEL_COL = "class"
LABEL_DICT = {'협박 대화': 0, '갈취 대화': 1, '직장 내 괴롭힘 대화': 2, '기타 괴롭힘 대화': 3, '일반 대화': 4}
ID_TO_LABEL = {v: k for k, v in LABEL_DICT.items()}
GPU_NUM = 0
TEST_FILE = "data/test.csv"

# 1. 데이터 로드 및 전처리
print("데이터 로드 중...")
df = pd.read_csv(TRAIN_FILE)

# 라벨 매핑
df[LABEL_COL] = df[LABEL_COL].map(LABEL_DICT)

# 데이터 분할
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42, stratify=df[LABEL_COL])
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df[LABEL_COL])

print(f"학습 데이터 크기: {len(train_df)}")
print(f"검증 데이터 크기: {len(val_df)}")
print(f"테스트 데이터 크기: {len(test_df)}")

# 클래스별 데이터 개수 확인
print("학습 데이터의 클래스 분포:")
print(train_df[LABEL_COL].value_counts())

# 2. Dataset 클래스 정의
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=True,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'token_type_ids': encoding['token_type_ids'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# 3. 데이터 로더 생성
def create_data_loader(df, tokenizer, max_len, batch_size):
    dataset = TextClassificationDataset(
        texts=df[TEXT_COL].values,
        labels=df[LABEL_COL].values,
        tokenizer=tokenizer,
        max_len=max_len
    )

    return DataLoader(
        dataset,
        batch_size=batch_size,
        num_workers=2
    )

# 4. 토크나이저 및 모델 로드
print(f"모델 '{MODEL_NAME}' 로드 중...")
tokenizer = ElectraTokenizer.from_pretrained(MODEL_NAME)
model = ElectraForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=len(LABEL_DICT))

# ELECTRA 인코더 부분 freeze
for param in model.electra.parameters():
    param.requires_grad = False

# classifier 부분만 학습 가능하게 설정
for param in model.classifier.parameters():
    param.requires_grad = True

# 학습 가능한 파라미터 확인 (디버깅용)
trainable_params = 0
all_param = 0
for name, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
        trainable_params += param.numel()
        print(f"✅ 학습 대상: {name}")
print(f"🔢 학습 대상 파라미터 수: {trainable_params:,} / 전체 파라미터 수: {all_param:,} ({trainable_params/all_param:.2%})")

# 5. 데이터 로더 생성
train_data_loader = create_data_loader(train_df, tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(val_df, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = create_data_loader(test_df, tokenizer, MAX_LEN, BATCH_SIZE)

# 6. 클래스 가중치 설정 (수동 조정) - 성능이 낮은 클래스에 더 높은 가중치 부여
# 클래스 가중치 수동 조정 - F1 점수가 낮은 클래스에 더 높은 가중치 부여
class_weights = [1.8, 1.8, 1.8, 2.0, 0.7]  # 기타 괴롭힘 대화에 적당한 가중치

print("수동 설정된 클래스별 가중치:")
for i, weight in enumerate(class_weights):
    print(f"클래스 {i} ({ID_TO_LABEL[i]}): {weight:.4f}")

# 7. 모델 학습 함수 (가중치 적용한 손실 함수 사용)
def train_model(model, data_loader, optimizer, scheduler, device, class_weights_tensor):
    model.train()
    losses = []

    progress_bar = tqdm(data_loader, desc="학습 중")

    for batch in progress_bar:
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            labels=labels
        )

        # 기본 손실 함수 대신 가중치가 적용된 손실 함수 사용
        logits = outputs.logits
        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights_tensor)
        loss = loss_fct(logits.view(-1, model.config.num_labels), labels.view(-1))

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()
        scheduler.step()

        losses.append(loss.item())
        progress_bar.set_postfix({"loss": f"{np.mean(losses):.4f}"})

    return np.mean(losses)

# 8. 모델 평가 함수
def evaluate_model(model, data_loader, device, class_weights_tensor=None):
    model.eval()
    losses = []
    predictions = []
    real_labels = []

    with torch.no_grad():
        for batch in tqdm(data_loader, desc="평가 중"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids
            )

            # 평가 시에도 동일한 가중치 적용
            if class_weights_tensor is not None:
                loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights_tensor)
                loss = loss_fct(outputs.logits.view(-1, model.config.num_labels), labels.view(-1))
            else:
                loss_fct = torch.nn.CrossEntropyLoss()
                loss = loss_fct(outputs.logits.view(-1, model.config.num_labels), labels.view(-1))

            _, preds = torch.max(outputs.logits, dim=1)

            losses.append(loss.item())
            predictions.extend(preds.detach().cpu().numpy())
            real_labels.extend(labels.detach().cpu().numpy())

    return np.mean(losses), predictions, real_labels

# 9. 혼동 행렬 시각화 함수
def plot_confusion_matrix(cm, classes):
    try:
        # 방법 1: 나눔 글꼴 설치 및 사용
        !apt-get update -qq
        !apt-get install fonts-nanum -qq
        plt.rc('font', family='NanumGothic')
    except:
        try:
            # 방법 2: 한글 레이블을 영어로 변환
            classes = ['Threat', 'Extortion', 'Workplace Harassment', 'Other Harassment', 'Normal Conversation']
        except:
            pass

    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
    plt.xlabel('예측값')
    plt.ylabel('실제값')
    plt.title('혼동 행렬')
    plt.savefig('confusion_matrix.png')
    plt.close()

# 10. 학습 메인 함수
def train():
    # GPU 사용 설정
    device = torch.device(f"cuda:{GPU_NUM}" if torch.cuda.is_available() else "cpu")
    print(f"사용 중인 디바이스: {device}")
    model.to(device)

    # 클래스 가중치 텐서를 디바이스로 이동
    class_weights_tensor = torch.FloatTensor(class_weights).to(device)

    # 옵티마이저 및 스케줄러 설정
    optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)

    # 전체 스텝 설정 - 여기서는 EPOCHS 전체를 사용
    total_steps = len(train_data_loader) * EPOCHS

    # 학습률 스케줄러 설정 - 더 느린 감소율 적용
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=0,
        num_training_steps=total_steps
    )

    # 학습 시작
    best_val_f1 = 0
    patience_counter = 0  # Early Stopping을 위한 카운터

    # 에폭별 성능 기록
    train_losses = []
    val_losses = []
    val_f1_scores = []
    learning_rates = []

    for epoch in range(EPOCHS):
        # 현재 학습률 가져오기
        current_lr = optimizer.param_groups[0]['lr']
        learning_rates.append(current_lr)

        print(f"\nEpoch {epoch + 1}/{EPOCHS} (학습률: {current_lr:.2e})")

        # 학습
        train_loss = train_model(model, train_data_loader, optimizer, scheduler, device, class_weights_tensor)
        train_losses.append(train_loss)
        print(f"Train Loss: {train_loss:.4f}")

        # 검증
        val_loss, val_predictions, val_labels = evaluate_model(model, val_data_loader, device, class_weights_tensor)
        val_losses.append(val_loss)
        val_f1 = f1_score(val_labels, val_predictions, average='macro')
        val_f1_scores.append(val_f1)

        print(f"Validation Loss: {val_loss:.4f}")
        print(f"Validation Macro F1: {val_f1:.4f}")

        # 클래스별 F1 점수 출력
        class_f1 = f1_score(val_labels, val_predictions, average=None)
        for i, score in enumerate(class_f1):
            print(f"클래스 {i} ({ID_TO_LABEL[i]}) F1: {score:.4f}")

        # 최고 성능 모델 저장 및 Early Stopping 로직
        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            torch.save(model.state_dict(), "best_model.pth")
            print(f"모델 저장됨: best_model.pth (F1: {val_f1:.4f})")
            patience_counter = 0  # 성능이 향상되었으므로 카운터 초기화
        else:
            patience_counter += 1  # 성능이 향상되지 않았으므로 카운터 증가
            print(f"성능 향상 없음: {patience_counter}/{EARLY_STOPPING_PATIENCE}")

            # Early Stopping 조건 확인
            if patience_counter >= EARLY_STOPPING_PATIENCE:
                print(f"\nEarly Stopping! {EARLY_STOPPING_PATIENCE}번의 에폭 동안 성능 향상 없음.")
                break

    # 학습 과정 시각화
    plt.figure(figsize=(12, 8))

    plt.subplot(2, 2, 1)
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Training and Validation Loss')

    plt.subplot(2, 2, 2)
    plt.plot(val_f1_scores, label='Validation F1')
    plt.xlabel('Epoch')
    plt.ylabel('F1 Score')
    plt.legend()
    plt.title('Validation F1 Score')

    plt.subplot(2, 2, 3)
    plt.plot(learning_rates, label='Learning Rate')
    plt.xlabel('Epoch')
    plt.ylabel('Learning Rate')
    plt.legend()
    plt.title('Learning Rate Schedule')

    plt.tight_layout()
    plt.savefig('training_history.png')
    plt.close()

    # 최고 성능 모델 로드
    model.load_state_dict(torch.load("best_model.pth"))

    # 테스트 세트 평가
    test_loss, test_predictions, test_labels = evaluate_model(model, test_data_loader, device, class_weights_tensor)
    test_f1 = f1_score(test_labels, test_predictions, average='macro')

    print("\n테스트 결과:")
    print(f"Test Loss: {test_loss:.4f}")
    print(f"Test Macro F1: {test_f1:.4f}")

    # 분류 보고서 출력
    class_names = [ID_TO_LABEL[i] for i in range(len(LABEL_DICT))]
    print("\n분류 보고서:")
    print(classification_report(test_labels, test_predictions, target_names=class_names))

    # 혼동 행렬 시각화
    cm = confusion_matrix(test_labels, test_predictions)
    plot_confusion_matrix(cm, class_names)

    return model

# 11. 실제 테스트 데이터 예측 및 제출 파일 생성
def predict_and_save():
    # 테스트 파일 로드
    test_data = pd.read_csv(TEST_FILE)

    # Dataset 생성
    class TestDataset(Dataset):
        def __init__(self, texts, tokenizer, max_len):
            self.texts = texts
            self.tokenizer = tokenizer
            self.max_len = max_len

        def __len__(self):
            return len(self.texts)

        def __getitem__(self, idx):
            text = str(self.texts[idx])

            encoding = self.tokenizer.encode_plus(
                text,
                add_special_tokens=True,
                max_length=self.max_len,
                return_token_type_ids=True,
                padding='max_length',
                truncation=True,
                return_attention_mask=True,
                return_tensors='pt'
            )

            return {
                'text': text,
                'input_ids': encoding['input_ids'].flatten(),
                'attention_mask': encoding['attention_mask'].flatten(),
                'token_type_ids': encoding['token_type_ids'].flatten(),
            }

    # 테스트 데이터 로더 생성
    test_dataset = TestDataset(
        texts=test_data['text'].values,
        tokenizer=tokenizer,
        max_len=MAX_LEN
    )

    test_data_loader = DataLoader(
        test_dataset,
        batch_size=BATCH_SIZE,
        num_workers=2
    )

    # GPU 설정
    device = torch.device(f"cuda:{GPU_NUM}" if torch.cuda.is_available() else "cpu")

    # 최고 성능 모델 로드
    model = ElectraForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=len(LABEL_DICT))
    model.load_state_dict(torch.load("best_model.pth"))
    model.to(device)

    # 예측
    model.eval()
    predictions = []

    with torch.no_grad():
        for batch in tqdm(test_data_loader, desc="테스트 예측 중"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids
            )

            _, preds = torch.max(outputs.logits, dim=1)
            predictions.extend(preds.detach().cpu().numpy())

    # 예측 결과 라벨로 변환
    label_predictions = [ID_TO_LABEL[p] for p in predictions]

    # 제출 파일 생성
    submission = pd.DataFrame({
        'idx': test_data['idx'],
        'class': label_predictions
    })

    submission.to_csv('submission.csv', index=False)
    print(f"제출 파일 생성 완료: submission.csv")

# 메인 실행
if __name__ == "__main__":
    # 모델 학습
    model = train()

    # 테스트 데이터 예측 및 제출 파일 생성
    predict_and_save()

데이터 로드 중...
학습 데이터 크기: 3477
검증 데이터 크기: 745
테스트 데이터 크기: 746
학습 데이터의 클래스 분포:
class
4    786
3    707
1    681
2    679
0    624
Name: count, dtype: int64
모델 'monologg/koelectra-base-v3-discriminator' 로드 중...


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ 학습 대상: classifier.dense.weight
✅ 학습 대상: classifier.dense.bias
✅ 학습 대상: classifier.out_proj.weight
✅ 학습 대상: classifier.out_proj.bias
🔢 학습 대상 파라미터 수: 594,437 / 전체 파라미터 수: 112,925,189 (0.53%)
수동 설정된 클래스별 가중치:
클래스 0 (협박 대화): 1.8000
클래스 1 (갈취 대화): 1.8000
클래스 2 (직장 내 괴롭힘 대화): 1.8000
클래스 3 (기타 괴롭힘 대화): 2.0000
클래스 4 (일반 대화): 0.7000
사용 중인 디바이스: cuda:0

Epoch 1/100 (학습률: 1.50e-05)


학습 중: 100%|██████████| 109/109 [00:06<00:00, 17.64it/s, loss=1.5838]


Train Loss: 1.5838


...

Validation Loss: 1.1679
Validation Macro F1: 0.5865
클래스 0 (협박 대화) F1: 0.4981
클래스 1 (갈취 대화) F1: 0.4729
클래스 2 (직장 내 괴롭힘 대화) F1: 0.5409
클래스 3 (기타 괴롭힘 대화) F1: 0.5055
클래스 4 (일반 대화) F1: 0.9152
성능 향상 없음: 5/5

Early Stopping! 5번의 에폭 동안 성능 향상 없음.


<ipython-input-6-89288e5b8766>:343: UserWarning: Glyph 8722 (\N{MINUS SIGN}) missing from font(s) NanumGothic.
  plt.savefig('training_history.png')
평가 중: 100%|██████████| 24/24 [00:01<00:00, 16.57it/s]



테스트 결과:
Test Loss: 1.1499
Test Macro F1: 0.6095

분류 보고서:
              precision    recall  f1-score   support

       협박 대화       0.54      0.57      0.55       134
       갈취 대화       0.58      0.40      0.48       146
 직장 내 괴롭힘 대화       0.58      0.50      0.54       145
   기타 괴롭힘 대화       0.48      0.64      0.55       152
       일반 대화       0.92      0.94      0.93       169

    accuracy                           0.62       746
   macro avg       0.62      0.61      0.61       746
weighted avg       0.63      0.62      0.62       746

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
테스트 예측 중: 100%|██████████| 16/16 [00:01<00:00, 14.85it/s]


제출 파일 생성 완료: submission.csv


#### 전체 모델 한번 학습하기
  - 현재는 ELECTRA 인코더를 고정하고 분류기만 학습하고 있는데, 이는 학습 데이터가 충분하다면 모델의 표현력을 제한할 수 있어 전체 모델을 미세 조정하는 방법을 시도 (F1 점수 높이기 위해서)
  - 비교/대조를 위해 이전 전체 모델 학습 환경에 맞춰 진행
    - 같은 결과 확인
- 클래스 불균형 해결
  - 클래스 가중치 적용: 클래스별 샘플 수에 반비례하는 가중치를 계산하여 적용
    - **성능 향상 있었지만, 아래에서 점수가 예상 이하**  
    https://www.kaggle.com/competitions/aiffel-dl-thon-dktc-online-13-a/submissions
    
    - 모델 개선 시 성능 향상 (점수 향상으로 이어지는) 있겠지만, 최종적으로 학습 데이터 (구성) 탓을 하게 됨

In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score, confusion_matrix
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import ElectraTokenizer, ElectraForSequenceClassification
from transformers import get_linear_schedule_with_warmup, get_cosine_schedule_with_warmup
from torch.optim import AdamW
from tqdm import tqdm
import random
import matplotlib.pyplot as plt
import seaborn as sns
import re
from functools import partial

# 시드 고정
def set_seed(seed_value=42):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)
    torch.backends.cudnn.deterministic = True

set_seed(42)

# 설정값
MODEL_NAME = "monologg/koelectra-base-v3-discriminator"
MAX_LEN = 128
BATCH_SIZE = 32
EPOCHS = 10
EARLY_STOPPING_PATIENCE = 2
LEARNING_RATE = 2e-5
TRAIN_FILE = "data/train_preprocessed_1.csv"
TEXT_COL = "text"
LABEL_COL = "class"
LABEL_DICT = {'협박 대화': 0, '갈취 대화': 1, '직장 내 괴롭힘 대화': 2, '기타 괴롭힘 대화': 3, '일반 대화': 4}
ID_TO_LABEL = {v: k for k, v in LABEL_DICT.items()}
GPU_NUM = 0
TEST_FILE = "data/test.csv"

# 1. 데이터 로드 및 전처리
print("데이터 로드 중...")
df = pd.read_csv(TRAIN_FILE)

# 라벨 매핑
df[LABEL_COL] = df[LABEL_COL].map(LABEL_DICT)

# 데이터 분할
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42, stratify=df[LABEL_COL])
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df[LABEL_COL])

print(f"학습 데이터 크기: {len(train_df)}")
print(f"검증 데이터 크기: {len(val_df)}")
print(f"테스트 데이터 크기: {len(test_df)}")

# 클래스별 데이터 개수 확인
print("학습 데이터의 클래스 분포:")
print(train_df[LABEL_COL].value_counts())

# 2. Dataset 클래스 정의
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=True,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'token_type_ids': encoding['token_type_ids'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# 3. 데이터 로더 생성
def create_data_loader(df, tokenizer, max_len, batch_size):
    dataset = TextClassificationDataset(
        texts=df[TEXT_COL].values,
        labels=df[LABEL_COL].values,
        tokenizer=tokenizer,
        max_len=max_len
    )

    return DataLoader(
        dataset,
        batch_size=batch_size,
        num_workers=2
    )

# 4. 토크나이저 및 모델 로드
print(f"모델 '{MODEL_NAME}' 로드 중...")
tokenizer = ElectraTokenizer.from_pretrained(MODEL_NAME)
model = ElectraForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=len(LABEL_DICT))

# 모든 파라미터 학습 가능하게 설정 (전체 모델 학습)
for param in model.parameters():
    param.requires_grad = True

# 학습 가능한 파라미터 확인 (디버깅용)
trainable_params = 0
all_param = 0
for name, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
        trainable_params += param.numel()
        print(f"✅ 학습 대상: {name}")
print(f"🔢 학습 대상 파라미터 수: {trainable_params:,} / 전체 파라미터 수: {all_param:,} ({trainable_params/all_param:.2%})")

# 5. 데이터 로더 생성
train_data_loader = create_data_loader(train_df, tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(val_df, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = create_data_loader(test_df, tokenizer, MAX_LEN, BATCH_SIZE)

# 6. 클래스 가중치 설정 (더 세밀하게 조정)
# 성능이 낮은 클래스의 가중치 증가, 성능이 높은 클래스의 가중치 감소
class_weights = [2.8, 2.0, 1.5, 3.8, 0.4]  # 협박↑↑, 갈취=, 직장내괴롭힘↓, 기타괴롭힘↑↑↑↑, 일반대화↓↓

print("수동 설정된 클래스별 가중치:")
for i, weight in enumerate(class_weights):
    print(f"클래스 {i} ({ID_TO_LABEL[i]}): {weight:.4f}")

# 7. 모델 학습 함수 (가중치 적용한 손실 함수 사용)
def train_model(model, data_loader, optimizer, scheduler, device, class_weights_tensor):
    model.train()
    losses = []

    progress_bar = tqdm(data_loader, desc="학습 중")

    for batch in progress_bar:
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            labels=labels
        )

        # 기본 손실 함수 대신 가중치가 적용된 손실 함수 사용
        logits = outputs.logits
        loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights_tensor)
        loss = loss_fct(logits.view(-1, model.config.num_labels), labels.view(-1))

        loss.backward()

        # 그래디언트 클리핑 설정 (전체 모델 학습시 안정성 향상)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()
        scheduler.step()

        losses.append(loss.item())
        progress_bar.set_postfix({"loss": f"{np.mean(losses):.4f}"})

    return np.mean(losses)

# 8. 모델 평가 함수
def evaluate_model(model, data_loader, device, class_weights_tensor=None):
    model.eval()
    losses = []
    predictions = []
    real_labels = []

    with torch.no_grad():
        for batch in tqdm(data_loader, desc="평가 중"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids
            )

            # 평가 시에도 동일한 가중치 적용
            if class_weights_tensor is not None:
                loss_fct = torch.nn.CrossEntropyLoss(weight=class_weights_tensor)
                loss = loss_fct(outputs.logits.view(-1, model.config.num_labels), labels.view(-1))
            else:
                loss_fct = torch.nn.CrossEntropyLoss()
                loss = loss_fct(outputs.logits.view(-1, model.config.num_labels), labels.view(-1))

            _, preds = torch.max(outputs.logits, dim=1)

            losses.append(loss.item())
            predictions.extend(preds.detach().cpu().numpy())
            real_labels.extend(labels.detach().cpu().numpy())

    return np.mean(losses), predictions, real_labels

# 9. 혼동 행렬 시각화 함수
def plot_confusion_matrix(cm, classes):
    try:
        # 방법 1: 나눔 글꼴 설치 및 사용
        !apt-get update -qq
        !apt-get install fonts-nanum -qq
        plt.rc('font', family='NanumGothic')
    except:
        try:
            # 방법 2: 한글 레이블을 영어로 변환
            classes = ['Threat', 'Extortion', 'Workplace Harassment', 'Other Harassment', 'Normal Conversation']
        except:
            pass

    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
    plt.xlabel('예측값')
    plt.ylabel('실제값')
    plt.title('혼동 행렬')
    plt.savefig('confusion_matrix.png')
    plt.close()

# 10. 학습 메인 함수
def train():
    # GPU 사용 설정
    device = torch.device(f"cuda:{GPU_NUM}" if torch.cuda.is_available() else "cpu")
    print(f"사용 중인 디바이스: {device}")
    model.to(device)

    # 클래스 가중치 텐서를 디바이스로 이동
    class_weights_tensor = torch.FloatTensor(class_weights).to(device)

    # 옵티마이저 설정
    optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)

    # 전체 스텝 설정
    total_steps = len(train_data_loader) * EPOCHS

    # 학습률 스케줄러
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=0,
        num_training_steps=total_steps
    )

    # 학습 시작
    best_val_f1 = 0
    patience_counter = 0  # Early Stopping을 위한 카운터

    # 에폭별 성능 기록
    train_losses = []
    val_losses = []
    val_f1_scores = []
    learning_rates = []

    for epoch in range(EPOCHS):
        # 현재 학습률 가져오기
        current_lr = optimizer.param_groups[0]['lr']
        learning_rates.append(current_lr)

        print(f"\nEpoch {epoch + 1}/{EPOCHS} (학습률: {current_lr:.2e})")

        # 학습
        train_loss = train_model(model, train_data_loader, optimizer, scheduler, device, class_weights_tensor)
        train_losses.append(train_loss)
        print(f"Train Loss: {train_loss:.4f}")

        # 검증
        val_loss, val_predictions, val_labels = evaluate_model(model, val_data_loader, device, class_weights_tensor)
        val_losses.append(val_loss)
        val_f1 = f1_score(val_labels, val_predictions, average='macro')
        val_f1_scores.append(val_f1)

        print(f"Validation Loss: {val_loss:.4f}")
        print(f"Validation Macro F1: {val_f1:.4f}")

        # 클래스별 F1 점수 출력
        class_f1 = f1_score(val_labels, val_predictions, average=None)
        for i, score in enumerate(class_f1):
            print(f"클래스 {i} ({ID_TO_LABEL[i]}) F1: {score:.4f}")

        # 최고 성능 모델 저장 및 Early Stopping 로직
        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            torch.save(model.state_dict(), "best_model.pth")
            print(f"모델 저장됨: best_model.pth (F1: {val_f1:.4f})")
            patience_counter = 0  # 성능이 향상되었으므로 카운터 초기화
        else:
            patience_counter += 1  # 성능이 향상되지 않았으므로 카운터 증가
            print(f"성능 향상 없음: {patience_counter}/{EARLY_STOPPING_PATIENCE}")

            # Early Stopping 조건 확인
            if patience_counter >= EARLY_STOPPING_PATIENCE:
                print(f"\nEarly Stopping! {EARLY_STOPPING_PATIENCE}번의 에폭 동안 성능 향상 없음.")
                break

    # 학습 과정 시각화
    plt.figure(figsize=(12, 8))

    plt.subplot(2, 2, 1)
    plt.plot(train_losses, label='Train Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Training and Validation Loss')

    plt.subplot(2, 2, 2)
    plt.plot(val_f1_scores, label='Validation F1')
    plt.xlabel('Epoch')
    plt.ylabel('F1 Score')
    plt.legend()
    plt.title('Validation F1 Score')

    plt.subplot(2, 2, 3)
    plt.plot(learning_rates, label='Learning Rate')
    plt.xlabel('Epoch')
    plt.ylabel('Learning Rate')
    plt.legend()
    plt.title('Learning Rate Schedule')

    plt.tight_layout()
    plt.savefig('training_history.png')
    plt.close()

    # 최고 성능 모델 로드
    model.load_state_dict(torch.load("best_model.pth"))

    # 테스트 세트 평가
    test_loss, test_predictions, test_labels = evaluate_model(model, test_data_loader, device, class_weights_tensor)
    test_f1 = f1_score(test_labels, test_predictions, average='macro')

    print("\n테스트 결과:")
    print(f"Test Loss: {test_loss:.4f}")
    print(f"Test Macro F1: {test_f1:.4f}")

    # 분류 보고서 출력
    class_names = [ID_TO_LABEL[i] for i in range(len(LABEL_DICT))]
    print("\n분류 보고서:")
    print(classification_report(test_labels, test_predictions, target_names=class_names))

    # 혼동 행렬 시각화
    cm = confusion_matrix(test_labels, test_predictions)
    plot_confusion_matrix(cm, class_names)

    return model

# 11. 실제 테스트 데이터 예측 및 제출 파일 생성
def predict_and_save():
    # 테스트 파일 로드
    test_data = pd.read_csv(TEST_FILE)

    # 전처리 함수 설정
    preprocess_fn = partial(
        preprocess,
        stopwords=None,   # 불용어 리스트 사용하지 않음
        speaker_token="",  # 발화 단위 토큰 사용하지 않음
        sep_token=" ", # 줄 구분 토큰
        use_silence=False, # 침묵 토큰 사용하지 않음
        repeat_limit=2 # 반복 문자 2개까지 허용
    )

    # 전처리 적용
    test_data['processed_text'] = test_data['text'].apply(preprocess_fn)

    # Dataset 생성
    class TestDataset(Dataset):
        def __init__(self, texts, tokenizer, max_len):
            self.texts = texts
            self.tokenizer = tokenizer
            self.max_len = max_len

        def __len__(self):
            return len(self.texts)

        def __getitem__(self, idx):
            text = str(self.texts[idx])

            encoding = self.tokenizer.encode_plus(
                text,
                add_special_tokens=True,
                max_length=self.max_len,
                return_token_type_ids=True,
                padding='max_length',
                truncation=True,
                return_attention_mask=True,
                return_tensors='pt'
            )

            return {
                'text': text,
                'input_ids': encoding['input_ids'].flatten(),
                'attention_mask': encoding['attention_mask'].flatten(),
                'token_type_ids': encoding['token_type_ids'].flatten(),
            }

    # 테스트 데이터 로더 생성 (전처리된 텍스트 사용)
    test_dataset = TestDataset(
        texts=test_data['processed_text'].values,
        tokenizer=tokenizer,
        max_len=MAX_LEN
    )

    test_data_loader = DataLoader(
        test_dataset,
        batch_size=BATCH_SIZE,
        num_workers=2
    )

    # GPU 설정
    device = torch.device(f"cuda:{GPU_NUM}" if torch.cuda.is_available() else "cpu")

    # 최고 성능 모델 로드
    model = ElectraForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=len(LABEL_DICT))
    model.load_state_dict(torch.load("best_model.pth"))
    model.to(device)

    # 예측
    model.eval()
    predictions = []

    with torch.no_grad():
        for batch in tqdm(test_data_loader, desc="테스트 예측 중"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids
            )

            _, preds = torch.max(outputs.logits, dim=1)
            predictions.extend(preds.detach().cpu().numpy())

    # 예측 결과 라벨로 변환
    label_predictions = [ID_TO_LABEL[p] for p in predictions]

    # 제출 파일 생성
    submission = pd.DataFrame({
        'idx': test_data['idx'],
        'class': label_predictions
    })

    submission.to_csv('submission.csv', index=False)
    print(f"제출 파일 생성 완료: submission.csv")

# 메인 실행
if __name__ == "__main__":
    # 모델 학습
    model = train()

    # 테스트 데이터 예측 및 제출 파일 생성
    predict_and_save()

데이터 로드 중...
학습 데이터 크기: 3477
검증 데이터 크기: 745
테스트 데이터 크기: 746
학습 데이터의 클래스 분포:
class
4    786
3    707
1    681
2    679
0    624
Name: count, dtype: int64
모델 'monologg/koelectra-base-v3-discriminator' 로드 중...


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ 학습 대상: electra.embeddings.word_embeddings.weight
✅ 학습 대상: electra.embeddings.position_embeddings.weight
✅ 학습 대상: electra.embeddings.token_type_embeddings.weight
✅ 학습 대상: electra.embeddings.LayerNorm.weight
✅ 학습 대상: electra.embeddings.LayerNorm.bias
✅ 학습 대상: electra.encoder.layer.0.attention.self.query.weight
✅ 학습 대상: electra.encoder.layer.0.attention.self.query.bias
✅ 학습 대상: electra.encoder.layer.0.attention.self.key.weight
✅ 학습 대상: electra.encoder.layer.0.attention.self.key.bias
✅ 학습 대상: electra.encoder.layer.0.attention.self.value.weight
✅ 학습 대상: electra.encoder.layer.0.attention.self.value.bias
✅ 학습 대상: electra.encoder.layer.0.attention.output.dense.weight
✅ 학습 대상: electra.encoder.layer.0.attention.output.dense.bias
✅ 학습 대상: electra.encoder.layer.0.attention.output.LayerNorm.weight
✅ 학습 대상: electra.encoder.layer.0.attention.output.LayerNorm.bias
✅ 학습 대상: electra.encoder.layer.0.intermediate.dense.weight
✅ 학습 대상: electra.encoder.layer.0.intermediate.dense.bias
✅ 학습 대상: electra.enco

학습 중: 100%|██████████| 109/109 [00:17<00:00,  6.15it/s, loss=1.2272]


Train Loss: 1.2272


...

Validation Loss: 0.5778
Validation Macro F1: 0.8845
클래스 0 (협박 대화) F1: 0.8193
클래스 1 (갈취 대화) F1: 0.8407
클래스 2 (직장 내 괴롭힘 대화) F1: 0.9379
클래스 3 (기타 괴롭힘 대화) F1: 0.8276
클래스 4 (일반 대화) F1: 0.9970
성능 향상 없음: 2/2

Early Stopping! 2번의 에폭 동안 성능 향상 없음.


<ipython-input-5-0a124382cd12>:341: UserWarning: Glyph 8722 (\N{MINUS SIGN}) missing from font(s) NanumGothic.
  plt.savefig('training_history.png')
평가 중: 100%|██████████| 24/24 [00:01<00:00, 16.86it/s]



테스트 결과:
Test Loss: 0.4856
Test Macro F1: 0.9076

분류 보고서:
              precision    recall  f1-score   support

       협박 대화       0.90      0.84      0.86       134
       갈취 대화       0.89      0.88      0.89       146
 직장 내 괴롭힘 대화       0.95      0.95      0.95       145
   기타 괴롭힘 대화       0.83      0.88      0.85       152
       일반 대화       0.98      0.99      0.98       169

    accuracy                           0.91       746
   macro avg       0.91      0.91      0.91       746
weighted avg       0.91      0.91      0.91       746

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
테스트 예측 중: 100%|██████████| 16/16 [00:01<00:00, 15.48it/s]

제출 파일 생성 완료: submission.csv


#### 토크나이저만 koRoBERTa로 대체 - 모델과 토크나이저를 분리해서 사용하는 방식
- roberta 토크나이저 로드 중에 에러가 발생
  - klue/roberta-base 모델이 RobertaTokenizer와 호환되지 않아 발생한 문제
  - klue/roberta-base는 실제로는 BERT 기반 토크나이저를 사용
- BERT 토크나이저를 사용하도록 코드를 수정
  - KoElectra 모델과 KLUE/roberta-base의 BERT 기반 토크나이저를 함께 사용
  (KoElectra 모델은 BERT 기반 토크나이저의 출력을 입력으로 받을 수 있게 되고, 두 시스템을 함께 사용 → 토크나이저와 모델 간의 호환성 문제를 해결하는 방법)
  - **결과 : 성능 저하 발생 (호환성 문제일 듯)**

In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score, confusion_matrix
import torch
from torch.utils.data import Dataset, DataLoader
# 토크나이저와 모델 import 수정
from transformers import ElectraForSequenceClassification, BertTokenizer
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm
import random
import matplotlib.pyplot as plt
import seaborn as sns

# 시드 고정
def set_seed(seed_value=42):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)
    torch.backends.cudnn.deterministic = True

set_seed(42)

# 설정값
ELECTRA_MODEL_NAME = "monologg/koelectra-base-v3-discriminator"
BERT_TOKENIZER_NAME = "klue/roberta-base"  # BERT 기반 토크나이저
MAX_LEN = 128
BATCH_SIZE = 32
EPOCHS = 10
SCHEDULER_EPOCHS = 5
EARLY_STOPPING_PATIENCE = 2
LEARNING_RATE = 2e-5
TRAIN_FILE = "data/train_preprocessed_1.csv"
TEXT_COL = "text"
LABEL_COL = "class"
LABEL_DICT = {'협박 대화': 0, '갈취 대화': 1, '직장 내 괴롭힘 대화': 2, '기타 괴롭힘 대화': 3, '일반 대화': 4}
ID_TO_LABEL = {v: k for k, v in LABEL_DICT.items()}
GPU_NUM = 0
TEST_FILE = "data/test.csv"

# 1. 데이터 로드 및 전처리
print("데이터 로드 중...")
df = pd.read_csv(TRAIN_FILE)

# 라벨 매핑
df[LABEL_COL] = df[LABEL_COL].map(LABEL_DICT)

# 데이터 분할
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42, stratify=df[LABEL_COL])
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df[LABEL_COL])

print(f"학습 데이터 크기: {len(train_df)}")
print(f"검증 데이터 크기: {len(val_df)}")
print(f"테스트 데이터 크기: {len(test_df)}")

# 클래스별 데이터 개수 확인
print("학습 데이터의 클래스 분포:")
print(train_df[LABEL_COL].value_counts())

# 2. Dataset 클래스 정의
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        # BERT 토크나이저는 token_type_ids를 반환함
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_token_type_ids=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'token_type_ids': encoding['token_type_ids'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# 3. 데이터 로더 생성
def create_data_loader(df, tokenizer, max_len, batch_size):
    dataset = TextClassificationDataset(
        texts=df[TEXT_COL].values,
        labels=df[LABEL_COL].values,
        tokenizer=tokenizer,
        max_len=max_len
    )

    return DataLoader(
        dataset,
        batch_size=batch_size,
        num_workers=2
    )

# 4. 토크나이저 및 모델 로드
print(f"BERT 토크나이저 '{ROBERTA_TOKENIZER_NAME}' 로드 중...")
tokenizer = BertTokenizer.from_pretrained(ROBERTA_TOKENIZER_NAME)

print(f"koElectra 모델 '{ELECTRA_MODEL_NAME}' 로드 중...")
model = ElectraForSequenceClassification.from_pretrained(ELECTRA_MODEL_NAME, num_labels=len(LABEL_DICT))

# 5. 데이터 로더 생성
train_data_loader = create_data_loader(train_df, tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(val_df, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = create_data_loader(test_df, tokenizer, MAX_LEN, BATCH_SIZE)

# 6. 모델 학습 함수
def train_model(model, data_loader, optimizer, scheduler, device):
    model.train()
    losses = []

    progress_bar = tqdm(data_loader, desc="학습 중")

    for batch in progress_bar:
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        labels = batch['labels'].to(device)

        # BERT 토크나이저는 token_type_ids를 항상 제공
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            labels=labels
        )

        loss = outputs.loss
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()
        scheduler.step()

        losses.append(loss.item())
        progress_bar.set_postfix({"loss": f"{np.mean(losses):.4f}"})

    return np.mean(losses)

# 7. 모델 평가 함수
def evaluate_model(model, data_loader, device):
    model.eval()
    losses = []
    predictions = []
    real_labels = []

    with torch.no_grad():
        for batch in tqdm(data_loader, desc="평가 중"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids,
                labels=labels
            )

            _, preds = torch.max(outputs.logits, dim=1)

            losses.append(outputs.loss.item())
            predictions.extend(preds.detach().cpu().numpy())
            real_labels.extend(labels.detach().cpu().numpy())

    return np.mean(losses), predictions, real_labels

# 8. 테스트 세트 예측 함수
def predict_test(model, data_loader, device):
    model.eval()
    predictions = []

    with torch.no_grad():
        for batch in tqdm(data_loader, desc="테스트 예측 중"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids
            )

            _, preds = torch.max(outputs.logits, dim=1)
            predictions.extend(preds.detach().cpu().numpy())

    return predictions

# 9. 혼동 행렬 시각화 함수
def plot_confusion_matrix(cm, classes):
    # 한글 폰트 설정
    import matplotlib.font_manager as fm

    # 시스템에 설치된 폰트 경로 확인
    # 리눅스 환경이면 'NanumGothic'이나 다른 한글 폰트 사용
    # 윈도우라면 'Malgun Gothic' 등 사용
    try:
        # 방법 1: 나눔 글꼴 설치 및 사용
        !apt-get update -qq
        !apt-get install fonts-nanum -qq
        plt.rc('font', family='NanumGothic')
    except:
        try:
            # 방법 2: 한글 레이블을 영어로 변환
            classes = ['Threat', 'Extortion', 'Workplace Harassment', 'Other Harassment', 'Normal Conversation']
        except:
            pass

    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
    plt.xlabel('예측값')
    plt.ylabel('실제값')
    plt.title('혼동 행렬')
    plt.savefig('confusion_matrix.png')
    plt.close()

# 10. 학습 메인 함수
def train():
    # GPU 사용 설정
    device = torch.device(f"cuda:{GPU_NUM}" if torch.cuda.is_available() else "cpu")
    print(f"사용 중인 디바이스: {device}")
    model.to(device)

    # 옵티마이저 및 스케줄러 설정
    from torch.optim import AdamW
    optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)
    total_steps = len(train_data_loader) * SCHEDULER_EPOCHS

    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=0,
        num_training_steps=total_steps
    )

    # 학습 시작
    best_val_f1 = 0
    patience_counter = 0  # Early Stopping을 위한 카운터

    for epoch in range(EPOCHS):
        print(f"\nEpoch {epoch + 1}/{EPOCHS}")

        # 학습
        train_loss = train_model(model, train_data_loader, optimizer, scheduler, device)
        print(f"Train Loss: {train_loss:.4f}")

        # 검증
        val_loss, val_predictions, val_labels = evaluate_model(model, val_data_loader, device)
        val_f1 = f1_score(val_labels, val_predictions, average='macro')

        print(f"Validation Loss: {val_loss:.4f}")
        print(f"Validation Macro F1: {val_f1:.4f}")

        # 최고 성능 모델 저장 및 Early Stopping 로직
        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            torch.save(model.state_dict(), "best_model.pth")
            print(f"모델 저장됨: best_model.pth (F1: {val_f1:.4f})")
            patience_counter = 0  # 성능이 향상되었으므로 카운터 초기화
        else:
            patience_counter += 1  # 성능이 향상되지 않았으므로 카운터 증가
            print(f"성능 향상 없음: {patience_counter}/{EARLY_STOPPING_PATIENCE}")

            # Early Stopping 조건 확인
            if patience_counter >= EARLY_STOPPING_PATIENCE:
                print(f"\nEarly Stopping! {EARLY_STOPPING_PATIENCE}번의 에폭 동안 성능 향상 없음.")
                break

    # 최고 성능 모델 로드
    model.load_state_dict(torch.load("best_model.pth"))

    # 테스트 세트 평가
    test_loss, test_predictions, test_labels = evaluate_model(model, test_data_loader, device)
    test_f1 = f1_score(test_labels, test_predictions, average='macro')

    print("\n테스트 결과:")
    print(f"Test Loss: {test_loss:.4f}")
    print(f"Test Macro F1: {test_f1:.4f}")

    # 분류 보고서 출력
    class_names = [ID_TO_LABEL[i] for i in range(len(LABEL_DICT))]
    print("\n분류 보고서:")
    print(classification_report(test_labels, test_predictions, target_names=class_names))

    # 혼동 행렬 시각화
    cm = confusion_matrix(test_labels, test_predictions)
    plot_confusion_matrix(cm, class_names)

    return model

# 11. 실제 테스트 데이터 예측 및 제출 파일 생성
def predict_and_save():
    # 테스트 파일 로드
    test_data = pd.read_csv(TEST_FILE)

    # 전처리 함수 불러오기 (전처리 함수 정의 필요)
    # 아래 함수는 실제 구현이 필요합니다
    def preprocess(text, stopwords=None, speaker_token="", sep_token=" ", use_silence=False, repeat_limit=2):
        # 실제 전처리 구현
        # 필요한 경우 이 부분에 전처리 로직을 구현하세요
        return text

    # 전처리 함수 설정
    from functools import partial
    preprocess_fn = partial(
        preprocess,
        stopwords=None,
        speaker_token="",
        sep_token=" ",
        use_silence=False,
        repeat_limit=2
    )

    # 전처리 적용
    test_data['processed_text'] = test_data['text'].apply(preprocess_fn)

    # Dataset 생성
    class TestDataset(Dataset):
        def __init__(self, texts, tokenizer, max_len):
            self.texts = texts
            self.tokenizer = tokenizer
            self.max_len = max_len

        def __len__(self):
            return len(self.texts)

        def __getitem__(self, idx):
            text = str(self.texts[idx])

            # BERT 토크나이저 포맷에 맞게 수정
            encoding = self.tokenizer.encode_plus(
                text,
                add_special_tokens=True,
                max_length=self.max_len,
                padding='max_length',
                truncation=True,
                return_token_type_ids=True,
                return_attention_mask=True,
                return_tensors='pt'
            )

            return {
                'text': text,
                'input_ids': encoding['input_ids'].flatten(),
                'attention_mask': encoding['attention_mask'].flatten(),
                'token_type_ids': encoding['token_type_ids'].flatten(),
            }

    # 테스트 데이터 로더 생성
    test_dataset = TestDataset(
        texts=test_data['processed_text'].values,
        tokenizer=tokenizer,
        max_len=MAX_LEN
    )

    test_data_loader = DataLoader(
        test_dataset,
        batch_size=BATCH_SIZE,
        num_workers=2
    )

    # GPU 설정
    device = torch.device(f"cuda:{GPU_NUM}" if torch.cuda.is_available() else "cpu")

    # 최고 성능 모델 로드
    model = ElectraForSequenceClassification.from_pretrained(ELECTRA_MODEL_NAME, num_labels=len(LABEL_DICT))
    model.load_state_dict(torch.load("best_model.pth"))
    model.to(device)

    # 예측
    model.eval()
    predictions = []

    with torch.no_grad():
        for batch in tqdm(test_data_loader, desc="테스트 예측 중"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            # token_type_ids 제거
            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )

            _, preds = torch.max(outputs.logits, dim=1)
            predictions.extend(preds.detach().cpu().numpy())

    # 예측 결과 라벨로 변환
    label_predictions = [str(p) for p in predictions]

    # 제출 파일 생성
    submission = pd.DataFrame({
        'idx': test_data['idx'],
        'class': label_predictions
    })

    submission.to_csv('data/submission.csv', index=False)
    print(f"제출 파일 생성 완료: submission.csv")

# 메인 실행
if __name__ == "__main__":
    # 모델 학습
    model = train()

    # 테스트 데이터 예측 및 제출 파일 생성
    predict_and_save()

데이터 로드 중...
학습 데이터 크기: 3477
검증 데이터 크기: 745
테스트 데이터 크기: 746
학습 데이터의 클래스 분포:
class
4    786
3    707
1    681
2    679
0    624
Name: count, dtype: int64
BERT 토크나이저 'klue/roberta-base' 로드 중...


vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

koElectra 모델 'monologg/koelectra-base-v3-discriminator' 로드 중...


config.json:   0%|          | 0.00/467 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/452M [00:00<?, ?B/s]

...

Validation Loss: 0.6966
Validation Macro F1: 0.7444
성능 향상 없음: 2/2

Early Stopping! 2번의 에폭 동안 성능 향상 없음.


평가 중: 100%|██████████| 24/24 [00:01<00:00, 17.11it/s]



테스트 결과:
Test Loss: 0.6758
Test Macro F1: 0.7623

분류 보고서:
              precision    recall  f1-score   support

       협박 대화       0.67      0.51      0.58       134
       갈취 대화       0.74      0.72      0.73       146
 직장 내 괴롭힘 대화       0.77      0.89      0.83       145
   기타 괴롭힘 대화       0.66      0.72      0.69       152
       일반 대화       0.98      0.99      0.98       169

    accuracy                           0.78       746
   macro avg       0.77      0.77      0.76       746
weighted avg       0.77      0.78      0.77       746

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
테스트 예측 중: 100%|██████████| 16/16 [00:01<00:00, 15.60it/s]


제출 파일 생성 완료: submission.csv


#### 모델이 모든 클래스에 대해 비등하게 학습시키기 위해 한번 과적합되는 일반 대화 수를 줄여봄
- 데이터프레임 가공
  - 일반 대화만 개수 조절
- 이전 모델 개발 적용 (코드 출력 확인 요망)

In [ ]:
import pandas as pd

# 데이터셋 로드
realize = pd.read_csv("data/train+normal_conversation.csv")

# 클래스별 개수 확인
print("처리 전 클래스별 개수:")
realize['class'].value_counts()

# 일반 대화 데이터의 첫 번째 인덱스 찾기
normal_conv_indices = realize[realize['class'] == '일반 대화'].index
first_normal_conv_index = normal_conv_indices[0] if len(normal_conv_indices) > 0 else None
print(f"일반 대화가 처음 시작하는 위치(인덱스): {first_normal_conv_index}")

# 유지할 일반 대화 개수 설정
keep_count = 800

# 유지할 인덱스와 제거할 인덱스 구분
keep_indices = normal_conv_indices[:keep_count]
remove_indices = normal_conv_indices[keep_count:]

# 제거할 인덱스 제외하고 새로운 데이터프레임 생성
df_filtered = realize.drop(remove_indices)

# 결과 확인
print("\n처리 후 클래스별 개수:")
print(df_filtered['class'].value_counts())

# 파일 저장
df_filtered.to_csv("data/train+normal_conversation_filtered.csv", index=False)
print(f"\n일반 대화 {keep_count}개만 유지한 데이터셋이 저장되었습니다.")

처리 전 클래스별 개수:
일반 대화가 처음 시작하는 위치(인덱스): 3846

처리 후 클래스별 개수:
class
기타 괴롭힘 대화      1011
갈취 대화           973
직장 내 괴롭힘 대화     970
협박 대화           892
일반 대화           800
Name: count, dtype: int64

일반 대화 800개만 유지한 데이터셋이 저장되었습니다.


In [ ]:
import os
import re
import numpy as np
import pandas as pd
from functools import partial
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score, confusion_matrix
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import ElectraTokenizer, ElectraForSequenceClassification#, AdamW
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm
import random
import matplotlib.pyplot as plt
import seaborn as sns

# 시드 고정
def set_seed(seed_value=42):
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)
    torch.backends.cudnn.deterministic = True

set_seed(42)

#--------------------------
# 데이터 전처리 함수들
#--------------------------

# 1. 텍스트 정제 함수들
def remove_stopwords(tokens, stopword_list):
    stopword_set = stopword_list if isinstance(stopword_list, set) else set(stopword_list)
    return [token for token in tokens if token not in stopword_set]

def clean_text(text):
    text = re.sub(r"[^\w\s가-힣]", " ", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

def normalize_repetitions(text, repeat_limit=2):
    # 문자 반복 (예: ㅋㅋㅋㅋ → ㅋㅋ)
    text = re.sub(r'(.)\1{2,}', lambda m: m.group(1) * repeat_limit, text)

    # 음절 반복 (예: 하하하하 → 하하)
    text = re.sub(r'((..))\\1{1,}', lambda m: m.group(1) * repeat_limit, text)

    return text

# 2. 텍스트 정제
def tokenize_and_clean_text(text, stopword_list=None, repeat_limit=2):
    text = normalize_repetitions(text, repeat_limit=repeat_limit)
    text = clean_text(text)
    tokens = text.split()
    if stopword_list:
        tokens = remove_stopwords(tokens, stopword_list)
    return tokens

# 3. 한줄 단위 전처리
def preprocess_conversation_lines(
    text,
    stopwords=None,
    use_silence=False,
    speaker_token="[UTTER]",
    repeat_limit=2
):
    lines = text.strip().split('\n')
    results = []

    for line in lines:
        if not line.strip():
            processed = ["[SILENCE]"] if use_silence else []
        else:
            processed = tokenize_and_clean_text(line, stopword_list=stopwords, repeat_limit=repeat_limit)
            if not processed:
                processed = ["[SILENCE]"] if use_silence else []
            else:
                processed = [speaker_token] + processed if speaker_token else processed

        if processed:
            results.append(" ".join(processed).strip())

    return results

# 4. 여러줄을 한 줄로 flatten 함수
def flatten_utterances(utterance_tokens_list, sep_token=" "):
    return sep_token.join(utterance_tokens_list).strip()

# 5. 전체 전처리 파이프라인
def preprocess(
    text,
    stopwords=None,
    speaker_token="[UTTER]",
    use_silence=True,
    sep_token=" ",
    repeat_limit=2
):
    """
    전체 전처리 통합 함수
    """
    utterance_tokens = preprocess_conversation_lines(
        text,
        stopwords=stopwords,
        use_silence=use_silence,
        speaker_token=speaker_token,
        repeat_limit=repeat_limit
    )
    return flatten_utterances(utterance_tokens, sep_token=sep_token)

# 결측치 및 중복 제거 함수
def clean_dataframe(df, is_train=True):
    df = df.dropna(subset=['text'])
    df = df.drop_duplicates(subset=['text'])
    if is_train:
        df = df.dropna(subset=['class'])
    df = df.reset_index(drop=True)
    return df

# CSV 파일 로드 함수
def load_csv_files(file_list, is_train=True):
    df_list = []

    for file_path in file_list:
        df = pd.read_csv(file_path)

        if is_train:
            if 'conversation' in df.columns:
                df = df.rename(columns={'conversation': 'text'})
        else:
            if 'conversation' in df.columns:
                df = df.rename(columns={'conversation': 'text'})
            if 'class' not in df.columns:
                df['class'] = pd.NA  # test에는 class가 없으므로 NaN 처리

        df_list.append(df)

    return pd.concat(df_list, ignore_index=True)

# 데이터프레임 준비 함수
def prepare_dataset(file_paths, preprocess_func, is_train=True):
    df = load_csv_files(file_paths, is_train=is_train)
    df = clean_dataframe(df, is_train=is_train)
    df['text'] = df['text'].apply(preprocess_func)
    df = clean_dataframe(df, is_train=is_train)
    return df

#--------------------------
# 모델 학습 설정
#--------------------------

# 설정값
MODEL_NAME = "monologg/koelectra-base-v3-discriminator"
MAX_LEN = 128
BATCH_SIZE = 32
EPOCHS = 10  # 5에서 10으로 변경
SCHEDULER_EPOCHS = 5  # 스케줄러용 에폭 (원래 설정 유지)
EARLY_STOPPING_PATIENCE = 2  # 연속 2번의 에폭 동안 성능이 향상되지 않으면 학습 중단
LEARNING_RATE = 2e-5
TRAIN_PATH = "data/train+normal_conversation_filtered.csv"
TEXT_COL = "text"#"conversation"x : 편리성 때문에
LABEL_COL = "class"
LABEL_DICT = {'협박 대화': 0, '갈취 대화': 1, '직장 내 괴롭힘 대화': 2, '기타 괴롭힘 대화': 3, '일반 대화': 4}
ID_TO_LABEL = {v: k for k, v in LABEL_DICT.items()}
GPU_NUM = 0
TEST_FILE = "data/test.csv"

# 1. 데이터 전처리 및 로드
print("데이터 전처리 및 로드 중...")

# 불용어 리스트 정의
stopwords = '이 있 하 것 들 그 되 수 이 보 않 없 나 주 아니 등 같 우리 때 년 가 한 지 오 네 야 아 아니 그럼 내가 너'.split()

# 전처리 함수 설정 - 줄 바꿈 표시 X, 불용어 X
preprocess_fn = partial(
    preprocess,
    stopwords=None,   # 불용어 리스트 사용하지 않음
    speaker_token="",  # 발화 단위 토큰 사용하지 않음
    sep_token=" ", # 줄 구분 시 토큰
    use_silence=False, # [SILENCE] 토큰 사용하지 않음
    repeat_limit=2 # 반복 문자 2개까지 허용
)

# 데이터셋 준비
train_files = [TRAIN_PATH]

# 데이터 전처리 및 저장
print("데이터 전처리 시작...")
train_df = prepare_dataset(train_files, preprocess_fn, is_train=True)
# train_df.to_csv("dataset/train_preprocessed_1.csv", index=False)
# print(f"✅ 전처리된 데이터 저장 완료: {train_df.shape}")
print(train_df.head())

# 라벨 매핑
train_df[LABEL_COL] = train_df[LABEL_COL].map(LABEL_DICT)

# 데이터 분할
train_data, temp_data = train_test_split(train_df, test_size=0.3, random_state=42, stratify=train_df[LABEL_COL])
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42, stratify=temp_data[LABEL_COL])

print(f"학습 데이터 크기: {len(train_data)}")
print(f"검증 데이터 크기: {len(val_data)}")
print(f"테스트 데이터 크기: {len(test_data)}")

# 클래스별 데이터 개수 확인
print("학습 데이터의 클래스 분포:")
print(train_data[LABEL_COL].value_counts())

# 2. Dataset 클래스 정의
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=True,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'token_type_ids': encoding['token_type_ids'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# 3. 데이터 로더 생성
def create_data_loader(df, tokenizer, max_len, batch_size):
    dataset = TextClassificationDataset(
        texts=df[TEXT_COL].values,
        labels=df[LABEL_COL].values,
        tokenizer=tokenizer,
        max_len=max_len
    )

    return DataLoader(
        dataset,
        batch_size=batch_size,
        num_workers=2
    )

# 4. 토크나이저 및 모델 로드
print(f"모델 '{MODEL_NAME}' 로드 중...")
tokenizer = ElectraTokenizer.from_pretrained(MODEL_NAME)
model = ElectraForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=len(LABEL_DICT))

# 5. 데이터 로더 생성
train_data_loader = create_data_loader(train_data, tokenizer, MAX_LEN, BATCH_SIZE)
val_data_loader = create_data_loader(val_data, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = create_data_loader(test_data, tokenizer, MAX_LEN, BATCH_SIZE)

# 6. 모델 학습 함수
def train_model(model, data_loader, optimizer, scheduler, device):
    model.train()
    losses = []

    progress_bar = tqdm(data_loader, desc="학습 중")

    for batch in progress_bar:
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            labels=labels
        )

        loss = outputs.loss
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()
        scheduler.step()

        losses.append(loss.item())
        progress_bar.set_postfix({"loss": f"{np.mean(losses):.4f}"})

    return np.mean(losses)

# 7. 모델 평가 함수
def evaluate_model(model, data_loader, device):
    model.eval()
    losses = []
    predictions = []
    real_labels = []

    with torch.no_grad():
        for batch in tqdm(data_loader, desc="평가 중"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids,
                labels=labels
            )

            _, preds = torch.max(outputs.logits, dim=1)

            losses.append(outputs.loss.item())
            predictions.extend(preds.detach().cpu().numpy())
            real_labels.extend(labels.detach().cpu().numpy())

    return np.mean(losses), predictions, real_labels

# 8. 테스트 세트 예측 함수
def predict_test(model, data_loader, device):
    model.eval()
    predictions = []

    with torch.no_grad():
        for batch in tqdm(data_loader, desc="테스트 예측 중"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids
            )

            _, preds = torch.max(outputs.logits, dim=1)
            predictions.extend(preds.detach().cpu().numpy())

    return predictions

# 9. 혼동 행렬 시각화 함수
def plot_confusion_matrix(cm, classes):
    # 한글 폰트 설정
    import matplotlib.font_manager as fm

    # 시스템에 설치된 폰트 경로 확인
    # 리눅스 환경이면 'NanumGothic'이나 다른 한글 폰트 사용
    # 윈도우라면 'Malgun Gothic' 등 사용
    try:
        # 방법 1: 나눔 글꼴 설치 및 사용
        !apt-get update -qq
        !apt-get install fonts-nanum -qq
        plt.rc('font', family='NanumGothic')
    except:
        try:
            # 방법 2: 한글 레이블을 영어로 변환
            classes = ['Threat', 'Extortion', 'Workplace Harassment', 'Other Harassment', 'Normal Conversation']
        except:
            pass

    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
    plt.xlabel('예측값')
    plt.ylabel('실제값')
    plt.title('혼동 행렬')
    plt.savefig('confusion_matrix.png')
    plt.close()

# 10. 학습 메인 함수
def train():
    # GPU 사용 설정
    device = torch.device(f"cuda:{GPU_NUM}" if torch.cuda.is_available() else "cpu")
    print(f"사용 중인 디바이스: {device}")
    model.to(device)

    # 옵티마이저 및 스케줄러 설정
    # AdamW 임포트 부분 수정
    # from transformers import AdamW - 이 부분이 오류 발생
    from torch.optim import AdamW  # 이렇게 수정
    optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)  # correct_bias 파라미터 제거
    total_steps = len(train_data_loader) * SCHEDULER_EPOCHS  # 원래 에폭(5) 기준으로 설정

    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=0,
        num_training_steps=total_steps
    )

    # 학습 시작
    best_val_f1 = 0
    patience_counter = 0  # Early Stopping을 위한 카운터

    for epoch in range(EPOCHS):
        print(f"\nEpoch {epoch + 1}/{EPOCHS}")

        # 학습
        train_loss = train_model(model, train_data_loader, optimizer, scheduler, device)
        print(f"Train Loss: {train_loss:.4f}")

        # 검증
        val_loss, val_predictions, val_labels = evaluate_model(model, val_data_loader, device)
        val_f1 = f1_score(val_labels, val_predictions, average='macro')

        print(f"Validation Loss: {val_loss:.4f}")
        print(f"Validation Macro F1: {val_f1:.4f}")

        # 최고 성능 모델 저장 및 Early Stopping 로직
        if val_f1 > best_val_f1:
            best_val_f1 = val_f1
            torch.save(model.state_dict(), "best_model.pth")
            print(f"모델 저장됨: best_model.pth (F1: {val_f1:.4f})")
            patience_counter = 0  # 성능이 향상되었으므로 카운터 초기화
        else:
            patience_counter += 1  # 성능이 향상되지 않았으므로 카운터 증가
            print(f"성능 향상 없음: {patience_counter}/{EARLY_STOPPING_PATIENCE}")

            # Early Stopping 조건 확인
            if patience_counter >= EARLY_STOPPING_PATIENCE:
                print(f"\nEarly Stopping! {EARLY_STOPPING_PATIENCE}번의 에폭 동안 성능 향상 없음.")
                break

    # 최고 성능 모델 로드
    model.load_state_dict(torch.load("best_model.pth"))

    # 테스트 세트 평가
    test_loss, test_predictions, test_labels = evaluate_model(model, test_data_loader, device)
    test_f1 = f1_score(test_labels, test_predictions, average='macro')

    print("\n테스트 결과:")
    print(f"Test Loss: {test_loss:.4f}")
    print(f"Test Macro F1: {test_f1:.4f}")

    # 분류 보고서 출력
    class_names = [ID_TO_LABEL[i] for i in range(len(LABEL_DICT))]
    print("\n분류 보고서:")
    print(classification_report(test_labels, test_predictions, target_names=class_names))

    # 혼동 행렬 시각화
    cm = confusion_matrix(test_labels, test_predictions)
    plot_confusion_matrix(cm, class_names)

    return model

# 11. 실제 테스트 데이터 예측 및 제출 파일 생성
def predict_and_save():
    # 테스트 파일 로드
    test_data = pd.read_csv(TEST_FILE)

    # 텍스트 전처리 적용 - 학습 데이터와 동일한 방식으로 전처리
    # 줄 바꿈 표시 X, 불용어 X 설정
    test_data['processed_text'] = test_data['text'].apply(
        lambda x: preprocess(
            x,
            stopwords=None,
            speaker_token="",
            sep_token=" ",
            use_silence=False,
            repeat_limit=2
        )
    )

    # Dataset 생성
    class TestDataset(Dataset):
        def __init__(self, texts, tokenizer, max_len):
            self.texts = texts
            self.tokenizer = tokenizer
            self.max_len = max_len

        def __len__(self):
            return len(self.texts)

        def __getitem__(self, idx):
            text = str(self.texts[idx])

            encoding = self.tokenizer.encode_plus(
                text,
                add_special_tokens=True,
                max_length=self.max_len,
                return_token_type_ids=True,
                padding='max_length',
                truncation=True,
                return_attention_mask=True,
                return_tensors='pt'
            )

            return {
                'text': text,
                'input_ids': encoding['input_ids'].flatten(),
                'attention_mask': encoding['attention_mask'].flatten(),
                'token_type_ids': encoding['token_type_ids'].flatten(),
            }

    # 테스트 데이터 로더 생성
    test_dataset = TestDataset(
        texts=test_data['processed_text'].values,
        tokenizer=tokenizer,
        max_len=MAX_LEN
    )

    test_data_loader = DataLoader(
        test_dataset,
        batch_size=BATCH_SIZE,
        num_workers=2
    )

    # GPU 설정
    device = torch.device(f"cuda:{GPU_NUM}" if torch.cuda.is_available() else "cpu")

    # 최고 성능 모델 로드
    model = ElectraForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=len(LABEL_DICT))
    model.load_state_dict(torch.load("best_model.pth"))
    model.to(device)

    # 예측
    model.eval()
    predictions = []

    with torch.no_grad():
        for batch in tqdm(test_data_loader, desc="테스트 예측 중"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            token_type_ids = batch['token_type_ids'].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids
            )

            _, preds = torch.max(outputs.logits, dim=1)
            predictions.extend(preds.detach().cpu().numpy())

    # 예측 결과 라벨로 변환 (숫자로 유지)
    label_predictions = [str(p) for p in predictions]

    # 제출 파일 생성
    submission = pd.DataFrame({
        'idx': test_data['idx'],
        'class': label_predictions
    })

    submission.to_csv('data/submission.csv', index=False)
    print(f"제출 파일 생성 완료: data/submission.csv")

# 메인 실행
if __name__ == "__main__":
    # 모델 학습
    model = train()

    # 테스트 데이터 예측 및 제출 파일 생성
    predict_and_save()

데이터 전처리 및 로드 중...
데이터 전처리 시작...
   idx      class                                               text
0    0      협박 대화  지금 너 스스로를 죽여달라고 애원하는 것인가 아닙니다 죄송합니다 죽을 거면 혼자 죽...
1    1      협박 대화  길동경찰서입니다 9시 40분 마트에 폭발물을 설치할거다 네 똑바로 들어 한번만 더 ...
2    2  기타 괴롭힘 대화  너 되게 귀여운거 알지 나보다 작은 남자는 첨봤어 그만해 니들 놀리는거 재미없어 지...
3    3      갈취 대화  어이 거기 예 너 말이야 너 이리 오라고 무슨 일 너 옷 좋아보인다 얘 돈 좀 있나...
4    4      갈취 대화  저기요 혹시 날이 너무 뜨겁잖아요 저희 회사에서 이 선크림 파는데 한 번 손등에 발...
학습 데이터 크기: 3251
검증 데이터 크기: 697
테스트 데이터 크기: 697
학습 데이터의 클래스 분포:
class
3    707
1    681
2    679
0    624
4    560
Name: count, dtype: int64
모델 'monologg/koelectra-base-v3-discriminator' 로드 중...


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


사용 중인 디바이스: cuda:0

Epoch 1/10


학습 중: 100%|██████████| 102/102 [00:16<00:00,  6.15it/s, loss=1.2269]


Train Loss: 1.2269


...

Validation Loss: 0.3847
Validation Macro F1: 0.8939
성능 향상 없음: 2/2

Early Stopping! 2번의 에폭 동안 성능 향상 없음.


평가 중: 100%|██████████| 22/22 [00:01<00:00, 16.47it/s]



테스트 결과:
Test Loss: 0.4258
Test Macro F1: 0.8950

분류 보고서:
              precision    recall  f1-score   support

       협박 대화       0.86      0.81      0.84       134
       갈취 대화       0.86      0.89      0.88       146
 직장 내 괴롭힘 대화       0.93      0.96      0.95       146
   기타 괴롭힘 대화       0.83      0.81      0.82       151
       일반 대화       0.99      1.00      1.00       120

    accuracy                           0.89       697
   macro avg       0.90      0.90      0.90       697
weighted avg       0.89      0.89      0.89       697

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
테스트 예측 중: 100%|██████████| 16/16 [00:01<00:00, 15.23it/s]

제출 파일 생성 완료: data/submission.csv


결과적으로 클래스별 균등하게 학습하는 것처럼 보이나 더 성능 떨어짐  
즉, 근본적으로 **구성한 학습 데이터셋 문제**이고, 나아가 **모델을 건드리는데 미숙한 본인 실력 문제**